In this demo, we try to see how using different embeddings might result in different optimized architectures whilst keeping other hyper parameters and data the same. 

In [1]:
import pennylane as qml
from pennylane import numpy as np
import networkx as nx
import os
import time

In [2]:
from qose.subarchitecture_tree_search import run_tree_architecture_search

In [3]:
    # Create a unique name for your experiment
    EXPERIMENT_NAME = 'VariousEmbeddings'

    # Create a directory to store the data
    if not os.path.exists('data'):
        os.mkdir('data/')

    data_path = f'data/{EXPERIMENT_NAME}'
    if not os.path.exists(data_path):
        os.mkdir(data_path)

In [13]:

    config = {'nqubits': 3,
             'embedding': 'E1',
              'min_tree_depth': 2,
              'max_tree_depth': 10,
              'prune_rate': 0.3,
              'prune_step': 2,
              'plot_trees': False,
              'data_set': 'moons',
              'nsteps': 8,
              'optim': qml.AdamOptimizer,
              'batch_sizes': [8],
              'n_samples': 1000,
              'learning_rates': [0.1],
              'save_frequency': 1,
              'save_path': data_path,
              'save_timing': False,
              'circuit_type':'schuld',
              'Tmax': [100, 100, 100],
              'inf_time':'timeit',
              'fill':'redundant', # or 'pad'
              'rate_type': 'batch_cost', # 'accuracy' or 'batch_cost'
              'readout_layer': 'one_hot',  #'one_hot' or 'weighted_neuron'
              }

In [7]:
possible_emb = ['E1', 'E2', 'E3','E4','E5','E6']
for key in possible_emb:
    config['embedding'] = key
    print("##########Embedding:", key)
    t_0_local = time.time()
    run_tree_architecture_search(config, "local")
    t_1_local = time.time()
    print("##########Time taken for",key," embedding: ", t_1_local - t_0_local )


##########Embedding: E1
Depth = 1
Depth = 2
Prune Tree
Current best architecture:  E1
max W: 1.0
weights: []
Grow Pruned Tree
Depth = 3
Grow Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Depth = 4
Prune Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Grow Pruned Tree
architecture with max W:  ROOT
max W: 0.0
weights:  []
##########Time taken for E1  embedding:  0.007697105407714844
##########Embedding: E2
Depth = 1
Depth = 2
Prune Tree
Current best architecture:  E1
max W: 1.0
weights: []
Grow Pruned Tree
Depth = 3
Grow Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Depth = 4
Prune Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Grow Pruned Tree
architecture with max W:  ROOT
max W: 0.0
weights:  []
##########Time taken for E2  embedding:  0.009233951568603516
##########Embedding: E3
Depth = 1
Depth = 2
Prune Tree
Current best architecture:  E1
max W: 1.0
weights: []
Grow Pruned Tree
Depth = 3
Grow Tree
Current best architecture:  

In [14]:
run_tree_architecture_search(config, "local")

Depth = 1
Depth = 2
Prune Tree
Current best architecture:  E1
max W: 1.0
weights: []
Grow Pruned Tree
Depth = 3
Grow Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Depth = 4
Prune Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Grow Pruned Tree
Depth = 5
Grow Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Depth = 6
Prune Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Grow Pruned Tree
Depth = 7
Grow Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Depth = 8
Prune Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
Grow Pruned Tree
Depth = 9
Grow Tree
Current best architecture:  ROOT
max W: 0.0
weights: []
architecture with max W:  ROOT
max W: 0.0
weights:  []
